Here, we show the process of symbollically deriving the rk coefficients for the 2/3step integrator that breaks the order barrier.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import scipy.optimize as opt

Defining symbols

In [8]:
w1, w2, a1, b1, a2, b2, c2, x1, x2, z1, z2, dt = sp.symbols('w1, w2, a1, b1, a2, b2, c2, x1, x2, z1, z2, dt')
a, b, c, d, e = sp.symbols('a, b, c, d, e ')
t, t0,  y = sp.symbols('t, t0, y')



Making an optimization function that can solve order conditions with complex constraint to derive complex coefficients

In [9]:
def optimize_path(num_steps, order, start, symbols, expr_list, targets_real = 0, targets_imag = 0, take_complex = 0, method = 'lm'):
    '''
    inputs:
    
    '''

            
    # declare 'dt' timestep symbol
    dt = sp.symbols('dt')

    w = expr_list
        
    # setup substituion variables for 'u + I*v' form
    subs_u = sp.symbols("u:" + str(len(symbols) ), real=True)
    subs_v = sp.symbols("v:" + str(len(symbols) ), real=True)

    # setup factorial list because want linearity
    if targets_real == 0:
        targets_real = [1/np.math.factorial(i+2) for i in range(num_steps-1)]
        targets_real.reverse()
    
    # substitute the coefficient expressions in 'w' with corresponding 'u + I*v'
    eq = []
    i = 0
    for expression in w:
        j = 0
        new_expr_real = expression

        for symbol in symbols:
            new_expr_real = new_expr_real.subs(symbol, subs_u[j] + sp.I*subs_v[j])


            j += 1
        new_expr_real_part = new_expr_real.expand().as_real_imag()[0] - targets_real[i]
        eq.append(new_expr_real_part)
        if i <2:
            new_expr_imag_part = new_expr_real.expand().as_real_imag()[1] 
            eq.append(new_expr_imag_part)

            #expr2_real_part = expr2_real.expand().as_real_imag()[0] - targets_imag[i]
            #eq.append(expr2_real_part)
        i += 1
        
    for i in range(8):
        eq.append(0)


   
    
    # create single list of all symbols used
    all_symbols = []
    all_symbols.extend(subs_u[:])
    all_symbols.extend(subs_v[:])  
    print(len(eq))
    
    # create jacobian matrix
    M = sp.zeros(len(eq),len(all_symbols))
    z = 0
    for i in range(len(eq)):
        for sym in all_symbols:
            M[z] = sp.diff(eq[i],sym)
            z += 1

    # turn original equation and jacobian matrix into numpy functions
    new_f = sp.lambdify([all_symbols], eq, modules='numpy')
    new_j = sp.lambdify([all_symbols], M, modules='numpy')
        
    # encapsulate the numpy functions into dummy functions
    def root_f(x):
        return new_f(x)
    def jac_f(x):
        return new_j(x)
    
    # solve for roots!
    sol = opt.root(root_f, start, jac=jac_f, method = method)
    sol.x
    print(root_f(sol.x))
    
    # solve for final weight and print out weights
    weights = np.array_split(sol.x, 2)
    '''
    final_real = 1
    final_imag = 0
    for i in range(num_steps-1):
        print('Step %d: %12.12f +  %12.12fi' % (i+1, weights[0][i], weights[1][i]))
        final_real -= weights[0][i]
        final_imag -= weights[1][i]
    
    # print final step
    print('Step %d: %12.12f +  %12.12fi' % (num_steps, final_real, final_imag))

    # combine all steps to return as output# 5-step 5-order
    print(weights)
    all_weights = np.zeros((2,num_steps+1))
    all_weights[:,:-1] = weights
    all_weights[0][-1] = final_real
    all_weights[1][-1] = final_imag
    '''
        
    return weights

Order conditions at various orders (Derived in a mathematica notebook separately)

In [10]:
eq1 = a1*w1 + b1*w1 + a2*w2 + b2*w2 + c2*w2
eq2 = (2*a2*(a1*w1 + b1*w1)*w2 + 2*b1*w1**2*x1 + 2*b2*w2*(a1*w1 + b1*w1 + w2*x2) + 2*c2*w2*(a1*w1 + b1*w1 + w2*z1 + w2*z2))/2.

In [11]:
eq31 = (12*a2*b1*w1**2*w2*x1 + 6*b2*w2*(2*b1*w1**2*x1 + 2*(a1*w1 + b1*w1)*w2*x2) +   6*c2*w2*(2*b1*w1**2*x1 + 2*(a1*w1 + b1*w1)*w2*z1 + 2*w2*(a1*w1 + b1*w1 + w2*x2)*z2))/12.
eq32 = (6*a2*(a1*w1 + b1*w1)**2*w2 + 6*b1*w1**3*x1**2 + 6*b2*w2*(a1*w1 + b1*w1 + w2*x2)**2 +   6*c2*w2*(a1*w1 + b1*w1 + w2*z1 + w2*z2)**2)/12.


In [12]:
eq41 = ((a2*(6*a1**3*w1**3 + 18*a1**2*b1*w1**3 + 18*a1*b1**2*w1**3 + 6*b1**3*w1**3)*w2)/6. + b1*w1**4*x1**3 + \
        (b2*w2*(6*a1**3*w1**3 + 6*b1**3*w1**3 + 18*b1**2*w1**2*w2*x2 + 18*b1*w1*w2**2*x2**2 + 6*w2**3*x2**3 + \
     18*a1**2*w1**2*(b1*w1 + w2*x2) + 9*a1*w1*(2*b1**2*w1**2 + 4*b1*w1*w2*x2 + 2*w2**2*x2**2)))/6. + 
        (c2*w2*(6*a1**3*w1**3 + 18*a1**2*b1*w1**3 + 18*a1*b1**2*w1**3 + 6*b1**3*w1**3 + 6*w2**3*z1**3 + 
     18*w2**3*z1**2*z2 + 18*w2**3*z1*z2**2 + 6*w2**3*z2**3 +   9*(2*a1**2*w1**2 + 4*a1*b1*w1**2 + 2*b1**2*w1**2)*(w2*z1 + w2*z2) + 
 9*(a1*w1 + b1*w1)*(2*w2**2*z1**2 + 4*w2**2*z1*z2 + 2*w2**2*z2**2)))/6.)/6.
eq42 = ((a2*w2*(b1*w1**3*x1**2 + (4*a1*b1*w1**3*x1 + 4*b1**2*w1**3*x1)/2.) +  b2*w2*(b1*w1**3*x1**2 + 
    ((2*a1**2*w1**2 + 4*a1*b1*w1**2 + 2*b1**2*w1**2)*w2*x2)/2. +   (4*b1**2*w1**3*x1 + 2*b1*w1*w2*(2*(a1*w1 + b1*w1) + 2*w1*x1)*x2 + 4*(a1*w1 + b1*w1)*w2**2*x2**2 + 
    4*a1*w1*(b1*w1**2*x1 + (a1*w1 + b1*w1)*w2*x2))/2.) +  c2*w2*(b1*w1**3*x1**2 + ((2*a1**2*w1**2 + 4*a1*b1*w1**2 + 2*b1**2*w1**2)*w2*z1)/2. + 
 (w2*(2*a1**2*w1**2 + 2*b1**2*w1**2 + 4*b1*w1*w2*x2 + 2*w2**2*x2**2 + 4*a1*w1*(b1*w1 + w2*x2))*z2)/2. + 
 (4*a1*b1*w1**3*x1 + 4*b1**2*w1**3*x1 + 4*(a1*w1 + b1*w1)*w2**2*z1**2 + 
 2*w2**2*(2*(a1*w1 + b1*w1) + 2*(a1*w1 + b1*w1 + w2*x2))*z1*z2 + 4*w2**2*(a1*w1 + b1*w1 + w2*x2)*z2**2 + 
  2*(2*b1*w1**2*x1*(w2*z1 + w2*z2) +  2*(a1*w1 + b1*w1)*((a1*w1 + b1*w1)*w2*z1 + w2*(a1*w1 + b1*w1 + w2*x2)*z2)))/2.))/2.)
eq43 = (6*b1*b2*w1**2*w2**2*x1*x2 + 3*c2*w2*(2*b1*w1**2*w2*x1*z1 + 2*w2*(b1*w1**2*x1 + (a1*w1 + b1*w1)*w2*x2)*z2))/6.

In [13]:
eq51 =(((a2*(24*a1**4*w1**4 + 96*a1**3*b1*w1**4 + 144*a1**2*b1**2*w1**4 + 96*a1*b1**3*w1**4 + 24*b1**4*w1**4)*w2)/24. + 
         b1*w1**5*x1**4 + (b2*w2*(24*a1**4*w1**4 + 24*b1**4*w1**4 + 96*b1**3*w1**3*w2*x2 + 
              144*b1**2*w1**2*w2**2*x2**2 + 96*b1*w1*w2**3*x2**3 + 24*w2**4*x2**4 + 96*a1**3*w1**3*(b1*w1 + w2*x2) + 
              72*a1**2*w1**2*(2*b1**2*w1**2 + 4*b1*w1*w2*x2 + 2*w2**2*x2**2) + 
              16*a1*w1*(6*b1**3*w1**3 + 18*b1**2*w1**2*w2*x2 + 18*b1*w1*w2**2*x2**2 + 6*w2**3*x2**3)))/24. + 
         (c2*w2*(24*a1**4*w1**4 + 96*a1**3*b1*w1**4 + 144*a1**2*b1**2*w1**4 + 96*a1*b1**3*w1**4 + 24*b1**4*w1**4 + 
              24*w2**4*z1**4 + 96*w2**4*z1**3*z2 + 144*w2**4*z1**2*z2**2 + 96*w2**4*z1*z2**3 + 24*w2**4*z2**4 + 
              16*(6*a1**3*w1**3 + 18*a1**2*b1*w1**3 + 18*a1*b1**2*w1**3 + 6*b1**3*w1**3)*(w2*z1 + w2*z2) + 
              36*(2*a1**2*w1**2 + 4*a1*b1*w1**2 + 2*b1**2*w1**2)*(2*w2**2*z1**2 + 4*w2**2*z1*z2 + 2*w2**2*z2**2) + 
             16*(a1*w1 + b1*w1)*(6*w2**3*z1**3 + 18*w2**3*z1**2*z2 + 18*w2**3*z1*z2**2 + 6*w2**3*z2**3)))/24.)/24.)
eq52 = ((a2*w2*(b1*w1**4*x1**3 + (18*a1**2*b1*w1**4*x1 + 36*a1*b1**2*w1**4*x1 + 18*b1**3*w1**4*x1)/6.) + 
      b2*w2*(b1*w1**4*x1**3 + ((6*a1**3*w1**3 + 18*a1**2*b1*w1**3 + 18*a1*b1**2*w1**3 + 6*b1**3*w1**3)*w2*x2)/6. + 
     (18*b1**3*w1**4*x1 + 3*b1**2*w1**2*w2*(6*(a1*w1 + b1*w1) + 12*w1*x1)*x2 + 
     3*b1*w1*w2**2*(12*(a1*w1 + b1*w1) + 6*w1*x1)*x2**2 + 18*(a1*w1 + b1*w1)*w2**3*x2**3 + 
    18*a1**2*w1**2*(b1*w1**2*x1 + (a1*w1 + b1*w1)*w2*x2) + 
  9*a1*w1*(4*b1**2*w1**3*x1 + 2*b1*w1*w2*(2*(a1*w1 + b1*w1) + 2*w1*x1)*x2 + 4*(a1*w1 + b1*w1)*w2**2*x2**2))
     /6.) + c2*w2*(b1*w1**4*x1**3 + ((6*a1**3*w1**3 + 18*a1**2*b1*w1**3 + 18*a1*b1**2*w1**3 + 6*b1**3*w1**3)*
      w2*z1)/6. + (w2*(6*a1**3*w1**3 + 6*b1**3*w1**3 + 18*b1**2*w1**2*w2*x2 + 18*b1*w1*w2**2*x2**2 + 
      6*w2**3*x2**3 + 18*a1**2*w1**2*(b1*w1 + w2*x2) +  9*a1*w1*(2*b1**2*w1**2 + 4*b1*w1*w2*x2 + 2*w2**2*x2**2))*z2)/6. + 
     (18*a1**2*b1*w1**4*x1 + 36*a1*b1**2*w1**4*x1 + 18*b1**3*w1**4*x1 + 18*(a1*w1 + b1*w1)*w2**3*z1**3 + 
      3*w2**3*(12*(a1*w1 + b1*w1) + 6*(a1*w1 + b1*w1 + w2*x2))*z1**2*z2 + 3*w2**3*(6*(a1*w1 + b1*w1) + 12*(a1*w1 + b1*w1 + w2*x2))*z1*z2**2 + 
       18*w2**3*(a1*w1 + b1*w1 + w2*x2)*z2**3 +  3*(3*(4*a1*b1*w1**3*x1 + 4*b1**2*w1**3*x1)*(w2*z1 + w2*z2) + 
     3*(2*a1**2*w1**2 + 4*a1*b1*w1**2 + 2*b1**2*w1**2)* ((a1*w1 + b1*w1)*w2*z1 + w2*(a1*w1 + b1*w1 + w2*x2)*z2)) + 
   3*(3*b1*w1**2*x1*(2*w2**2*z1**2 + 4*w2**2*z1*z2 + 2*w2**2*z2**2) +  3*(a1*w1 + b1*w1)*(4*(a1*w1 + b1*w1)*w2**2*z1**2 +
        2*w2**2*(2*(a1*w1 + b1*w1) + 2*(a1*w1 + b1*w1 + w2*x2))*z1*z2 + 4*w2**2*(a1*w1 + b1*w1 + w2*x2)*z2**2)))/6.))/6.)
eq53 = ((a2*w2*(6*a1*b1*w1**4*x1**2 + 6*b1**2*w1**4*x1**2) + b2*w2*(6*b1**2*w1**4*x1**2 + 2*b1*w1*w2*
      ((3*(2*a1**2*w1**2 + 4*a1*b1*w1**2 + 2*b1**2*w1**2))/2. + 3*w1**2*x1**2)*x2 + 3*(2*a1**2*w1**2 + 4*a1*b1*w1**2 + 2*b1**2*w1**2)*w2**2*x2**2 + 
      6*a1*w1*(b1*w1**3*x1**2 + ((2*a1**2*w1**2 + 4*a1*b1*w1**2 + 2*b1**2*w1**2)*w2*x2)/2.)) + 
       c2*w2*(6*a1*b1*w1**4*x1**2 + 6*b1**2*w1**4*x1**2 + 3*(2*a1**2*w1**2 + 4*a1*b1*w1**2 + 2*b1**2*w1**2)*w2**2*z1**2 + 
     2*w2**2*((3*(2*a1**2*w1**2 + 4*a1*b1*w1**2 + 2*b1**2*w1**2))/2. + 
     (3*(2*a1**2*w1**2 + 2*b1**2*w1**2 + 4*b1*w1*w2*x2 + 2*w2**2*x2**2 + 4*a1*w1*(b1*w1 + w2*x2)))/2.)*z1*z2\
     + 3*w2**2*(2*a1**2*w1**2 + 2*b1**2*w1**2 + 4*b1*w1*w2*x2 + 2*w2**2*x2**2 + 4*a1*w1*(b1*w1 + w2*x2))*
      z2**2 + 2*(3*b1*w1**3*x1**2*(w2*z1 + w2*z2) + 3*(a1*w1 + b1*w1)*(((2*a1**2*w1**2 + 4*a1*b1*w1**2 + 2*b1**2*w1**2)*w2*z1)/2. + 
      (w2*(2*a1**2*w1**2 + 2*b1**2*w1**2 + 4*b1*w1*w2*x2 + 2*w2**2*x2**2 + 4*a1*w1*(b1*w1 + w2*x2))*z2)/2.)) ))/12.)
eq54 = ((2*a2*b1**2*w1**4*w2*x1**2 + b2*w2*(2*w2*(b1*w1**3*x1**2 + (4*a1*b1*w1**3*x1 + 4*b1**2*w1**3*x1)/2.)*x2 + 
 (4*b1**2*w1**4*x1**2 + 8*a1*b1*w1**3*w2*x1*x2 + 2*b1*w1*w2*(4*b1*w1**2*x1 + 4*w1*(a1*w1 + b1*w1)*x1)*x2 + 
  w2**2*(2*(2*a1**2*w1**2 + 4*a1*b1*w1**2 + 2*b1**2*w1**2) + 8*b1*w1**2*x1)*x2**2)/2.) + 
 c2*w2*(2*(w2*(b1*w1**3*x1**2 + (4*a1*b1*w1**3*x1 + 4*b1**2*w1**3*x1)/2.)*z1 + 
 w2*(b1*w1**3*x1**2 + ((2*a1**2*w1**2 + 4*a1*b1*w1**2 + 2*b1**2*w1**2)*w2*x2)/2. + 
 (4*b1**2*w1**3*x1 + 2*b1*w1*w2*(2*(a1*w1 + b1*w1) + 2*w1*x1)*x2 + 4*(a1*w1 + b1*w1)*w2**2*x2**2 + 
4*a1*w1*(b1*w1**2*x1 + (a1*w1 + b1*w1)*w2*x2))/2.)*z2) + 
 (4*b1**2*w1**4*x1**2 + w2**2*(2*(2*a1**2*w1**2 + 4*a1*b1*w1**2 + 2*b1**2*w1**2) + 8*b1*w1**2*x1)*z1**2 + 
 2*w2**2*(4*b1*w1**2*x1 + 4*(a1*w1 + b1*w1)*(a1*w1 + b1*w1 + w2*x2) + 4*(b1*w1**2*x1 + (a1*w1 + b1*w1)*w2*x2))*z1*z2 + 
     w2**2*(8*(b1*w1**2*x1 + (a1*w1 + b1*w1)*w2*x2) + 
   2*(2*a1**2*w1**2 + 2*b1**2*w1**2 + 4*b1*w1*w2*x2 + 2*w2**2*x2**2 + 4*a1*w1*(b1*w1 + w2*x2)))*z2**2 + 
   2*(4*b1*w1**2*x1*((a1*w1 + b1*w1)*w2*z1 + w2*(a1*w1 + b1*w1 + w2*x2)*z2) + 
      2*(a1*w1 + b1*w1)*(2*b1*w1**2*w2*x1*z1 + 2*w2*(b1*w1**2*x1 + (a1*w1 + b1*w1)*w2*x2)*z2)))/2.))/4.)
eq55 = b1*c2*w1**2*w2**3*x1*x2*z2

Now, we can try to find the parameters that solve these equations. Changing start changes the initial condition of the optimization process.

In [14]:
num_steps = 5
list_math = [ ]
order = 5
start = np.array([ 0.19254112,  0.31159223,  0.19917115, -0.14748027,  0.44410135,0,-0.36438814,  0.18248485, -0.23545467,  0.65551491,  0.4236295,  0 ])
start = np.array([0.28492564, 0.14650833, 0.07066997, 0.47322386, 0.02467217, -0.13816636,  0.38885579, -0.28296455, -0.02434782,  0.74553616])
start = [ 5.49178222e-01,  1.28569194e+00, -4.77500517e-01,  1.36543699e+00,
        0, 0,  8.17361099e-02,  2.50326653e-01,
        -6.75868234e-04,  8.83070159e-01, -8.41290458e-01, 6.62552778e-02,  4.29213564e+00, -5.79333251e-01,  7.85960306e-01,
         6.33947868e+00, -6.81879351e+00,  3.55809139e-01,  1.74455588e-01,
        -6.68500581e-04, -1.62664801e+00,  1.60948401e+00]
targets_real = [1, 1/2, 1/6,1/6,1/24,1/6 ,1/24, 1/120, 7/120, 1/30, 11/120, 1/120]
take_complex = [ 0, 0, 0, 0, 0, 0, 0,0,0,0,0,0]

symbols = [w1, w2, a1, b1, a2, b2, c2, x1, x2, z1, z2]
expr_list = [eq1, eq2, eq31, eq32, eq41, eq42, eq43, eq51, eq52, eq53, eq54, eq55] 

#opti = optimize_path(num_steps,order,start)
opti = optimize_path(num_steps,order,start, symbols, expr_list, targets_real,targets_imag, take_complex, 'lm')
#lib_add('Euler 2-step 2-order',opti)

22
[-2.886579864025407e-15, -3.3306690738754696e-16, -2.6645352591003757e-15, 2.220446049250313e-16, -2.7755575615628914e-16, 1.7486012637846216e-15, 1.1839140778846513e-13, -2.4480417692984702e-14, -4.163336342344337e-17, 1.0795548482933626e-12, 1.237274172005698e-13, 2.976785484776201e-15, 7.230327447871332e-15, 6.938893903907228e-18, 0, 0, 0, 0, 0, 0, 0, 0]


In [15]:
opti
#[0.28492564, 0.14650833, 0.07066997, 0.47322386, 0.02467217,-0.13816636,  0.38885579, -0.28296455, -0.02434782,  0.74553616]

[array([ 2.06599475e-01, -1.54492335e+00, -2.82496127e-01,  6.93078863e-01,
         4.13784970e-02, -1.44406938e-01,  8.32197831e-02,  6.04706648e-01,
        -9.27295228e-04,  9.96416253e-01, -9.42250087e-01]),
 array([ 4.28100857e-01,  7.18895570e+00, -1.19832732e+00,  4.49954556e-01,
         6.63477799e+00, -6.72748439e+00,  1.42260211e-02, -8.04650472e-01,
        -1.32411979e-03, -1.49186906e+00,  1.47131727e+00])]

These are the parameters we find

In [17]:
subs_list = []
for i in range(11):
    subs_list += [(symbols[i], opti[0][i]+sp.I*opti[1][i])]
subs_list

[(w1, 0.206599475221461 + 0.428100856781222*I),
 (w2, -1.54492335208623 + 7.18895570467947*I),
 (a1, -0.282496127265484 - 1.19832732325366*I),
 (b1, 0.693078863188054 + 0.449954555731326*I),
 (a2, 0.0413784970387994 + 6.63477799249045*I),
 (b2, -0.144406938041957 - 6.72748438879053*I),
 (c2, 0.0832197830868657 + 0.0142260210906774*I),
 (x1, 0.604706648448803 - 0.804650471556979*I),
 (x2, -0.000927295227575114 - 0.00132411979110159*I),
 (z1, 0.996416253292974 - 1.49186905836381*I),
 (z2, -0.942250086591574 + 1.47131726842163*I)]

We can plug these parameters into any of the order conditions to see if it is satisfied. For eq54, the real part of order condition must be $11/120 =0.091666$

In [20]:
sp_eq =eq54
e54=  sp_eq.subs(subs_list)
e54.expand()

0.0916666666666656 + 0.122537762660728*I

Try the parameters on a function and symbollically see if it has 5th order accuracy (in the real part)

In [205]:
def f(x):
    return x**2
y0 = 1
k11 = f(y0)
k12 = f(y0+ x1*w1*dt*k11)
y1 = y0 +a1*w1*dt*k11+b1*w1*dt*k12
k21 = f(y1)
k22 = f(y1 + x2*w2*dt*k21)
k23 = f(y1 + z1*w2*dt*k21 + z2*w2*dt*k22)
y2 = y1 +a2*w2*dt*k21+b2*w2*dt*k22+c2*w2*dt*k23
y2 = y2 + sp.O(dt**6)
(y2.subs(subs_list)).expand()

1 - 1.77635683940025e-15*I*dt + 1.0*dt - 1.77635683940025e-15*I*dt**2 + 1.00000000000001*dt**2 - 0.279083463558177*I*dt**3 + 0.999999999999957*dt**3 + 0.411112356770277*I*dt**4 + 1.00000000000011*dt**4 + 1.27992773284175*I*dt**5 + 0.999999999999855*dt**5 + O(dt**6)

In [206]:
exp_list = [x1*w1, a1*w1, b1*w1, x2*w2, z1*w2, z2*w2, a2*w2, b2*w2, c2*w2]
exp_list_v =  [complex((v.subs(subs_list)).expand()) for v in exp_list]#[a_121, b_11, b_12, a_221, a_231, a_232, b_21, b_22, b_23]

In [232]:
a_121, b_11, b_12, a_221, a_231, a_232, b_21, b_22, b_23 = exp_list_v 
exp_list_v

[(0.4694036325154083+0.09263506914186012j),
 (0.45464140214409554-0.3685106302474753j),
 (-0.04943620139945573+0.3896680302353586j),
 (0.01095163857727765-0.004620620729965784j),
 (9.185593839648694+9.468015654867008j),
 (-9.121530507932487-9.046866541549512j),
 (-47.76105170474552-9.95275527416814j),
 (48.58668492572205+9.355312652006003j),
 (-0.2308384217211647+0.5762852221742528j)]

In [179]:
def f(x):
    return x**2
y0 = 1
k11 = f(y0)
k12 = f(y0+ a_121*dt*k11)
y1 = y0 +b_11*dt*k11+b_12*dt*k12
k21 = f(y1)
k22 = f(y1 + a_221*dt*k21)
k23 = f(y1 + a_231*dt*k21 + a_232*dt*k22)
y2 = y1 +b_21*dt*k21+b_22*dt*k22+b_23*dt*k23
y2 = y2 + sp.O(dt**6)
y2.expand() #(y2.subs(subs_list)).expand()

1 - 8.88178419700125e-16*I*dt + 1.00000000000001*dt + 7.105427357601e-15*I*dt**2 + 1.0*dt**2 - 0.169764409723034*I*dt**3 + 0.999999999999959*dt**3 - 0.0842128812546147*I*dt**4 + 0.999999999999809*dt**4 + 0.0962579517640259*I*dt**5 + 1.00000000000012*dt**5 + O(dt**6)

A previously derived (stable) set of rk coefficients

In [2]:
a_121, b_11, b_12, a_221, a_231, a_232, b_21, b_22, b_23 = [(0.4694036325154083+0.09263506914186012j),
 (0.45464140214409554-0.3685106302474753j),
 (-0.04943620139945573+0.3896680302353586j),
 (0.01095163857727765-0.004620620729965784j),
 (9.185593839648694+9.468015654867008j),
 (-9.121530507932487-9.046866541549512j),
 (-47.76105170474552-9.95275527416814j),
 (48.58668492572205+9.355312652006003j),
 (-0.2308384217211647+0.5762852221742528j)]

The RK2/3 function

In [3]:
def rk5step(y0, f, dt):
    k11 = f(y0)
    k12 = f(y0+ a_121*dt*k11)
    y1 = y0 +b_11*dt*k11+b_12*dt*k12
    k21 = f(y1)
    k22 = f(y1 + a_221*dt*k21)
    k23 = f(y1 + a_231*dt*k21 + a_232*dt*k22)
    y2 = y1 +b_21*dt*k21+b_22*dt*k22+b_23*dt*k23
    return np.real(y2)

See if it has 5th order numerical accuracy

In [4]:
def f1(y):
    f1.counter  += 1
    return y
f1.counter = 0
error_lin = []
dt_lin = []
for N in [11, 51,  101,501, 1001]:
    end = 5
    y = np.zeros(N)
    y[0] = 1
    t = np.linspace(0, end, N)
    dt = end/(N-1)
    for i in range(1,N):
        y[i] = rk5step(y[i-1], f1, dt)  
    error = np.linalg.norm(y - np.exp(t), np.Inf)
    error_lin += [error]
    dt_lin += [dt]
error_lin

[0.02102129098480532,
 9.460583214604412e-06,
 3.0856827493153105e-07,
 1.0271605788148008e-10,
 3.950617610826157e-12]